# Entrega del modelo (local)

In [ ]:
import mlflow
import pandas as pd

Para servir el modelo en local con `MLflow` se puede seguir [esta guía](https://mlflow.org/docs/latest/deployment/deploy-model-locally.html). Es importante que se inicie el `tracking` de los recursos de `MLflow` en el directorio adecuado (la carpeta `mlflow`, donde están las carpetas de `mlrun` y `mlartifacts`) y que se inicie la interfaz gráfica (`mlflow ui`) para que se puedan cargar los modelos.

In [ ]:
%%bash
if [ pwd != 'C:\Users\valde\Desktop\Airbnb_Madrid\mlflow' ]
then
    cd mlflow/
fi

Primero se comprueba que se puede cargar el modelo de `MLflow` especificando el nombre y su versión y realizando predicciones `offline` por lotes:

In [ ]:
# URL donde se trackea el registro de recursos
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Para cargar el modelo hace falta especificar una de estas dos opciones:
# - 'runs:/<run_id>/<artifact_path>'
# - 'models:/<model_name>/<model_version>'
logged_model = 'models:/decision_tree/1'

# Este comando es por si acaso para instalar las dependencias del modelo especificado con 'pip'
mlflow.pyfunc.get_model_dependencies(logged_model, 'pip')

# Carga el modelo de 'Decision Tree' como objeto 'PyFuncModel'
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predicción con un ejemplo 'raw' (el formato de entrada es un 'dataframe', ya sea de una fila o N filas)
loaded_model.predict(pd.DataFrame(columns = ["square_feet", "accommodates", "cleaning_fee", "beds", "availability_30", "bedrooms", "availability_60", "availability_90", "availability_365", "security_deposit", "bathrooms", "guests_included", "neighbourhood", "neighbourhood_group_cleansed", "property_type", "room_type", "amenities", "X", "Y", "Z", "date"],
                                  data= [[5.46, 6, 60.0, 4.0, 30, 2.0, 60, 88, 359, 600.0, 2.0, 4, "Retiro", "Retiro", "Apartment", "Entire home/apt", 18, 4846242.12107298, -312605.06587381434, 4134833.3663519137, 2]]))

Ahora se prueba a servir el modelo con `Flask` en un `endpoint` de un `servicio web` local en el `puerto 1234`:

In [ ]:
# Se establece la variable de entorno para trackear la URL donde el Registro de modelos reside
%env MLFLOW_TRACKING_URI=http://localhost:5000

# Se sirve el modelo de producción desde el Registro de modelos, con un puerto distinto al que usa MLflow sin entorno Conda
!mlflow models serve -m "models:/decision_tree/1" -p 1234 #--enable-mlserver

El servidor de inferencia tiene 4 `endpoints`:

- `/invocations`: Un `endpoint` que acepta peticiones con datos de entrada y que devuelve las predicciones (`POST`).
- `/ping`: Para realizar comprobaciones de funcionamiento (`GET`).
- `/health`: Lo mismo que en el caso anterior (`GET`).
- `/version`: Devuelve la versión de `MLflow` (`GET`).

La celda e código anterior está permanentemente en ejecución, ya que el modelo está siendo `servido` en el `puerto 1234` del `localhost`, es decir, en la URL `http://127.0.0.1:1234/`. Por tanto, ahora se puede hacer un `curl` a esta dirección para hacer predicciones o ejecutar un `script` de Python para realizar la inferencia con una solicitud `POST` a la `URL` especificada anteriormente. El `script` sería algo parecido a esto:

In [ ]:
# Librerías
import requests
import json
import numpy as np

# URL local donde se sirven las predicciones del modelo
url = 'http://127.0.0.1:1234/invocations'

# Header para contenido de tipo JSON
headers = {'Content-type': 'application/json'}

# Datos a los que realizar la inferencia. MUY IMPORTANTE: Seguir el esquema de 'dataframe_split' con 'columns' y 'data'
X_test = json.dumps({
    "dataframe_split": {
        "columns": ["square_feet", "accommodates", "cleaning_fee", "beds", "availability_30", "bedrooms", "availability_60", "availability_90", "availability_365", "security_deposit", "bathrooms", "guests_included", "neighbourhood", "neighbourhood_group_cleansed", "property_type", "room_type", "amenities", "X", "Y", "Z", "date"], 
        "data": [
            [np.nan, 6, 60.0, 4.0, 30, 2.0, 60, 88, 359, 600.0, 2.0, 4, "Retiro", "Retiro", "Apartment", "Entire home/apt", 18, 4846242.12107298, -312605.06587381434, 4134833.3663519137, 2], 
            [np.nan, 6, 60.0, 4.0, 30, 2.0, 60, 88, 359, 600.0, 2.0, 4, "Retiro", "Retiro", "Apartment", "Entire home/apt", 18, 4846242.12107298, -312605.06587381434, 4134833.3663519137, 2]
            ]
    }
})                          

# Solicitud 'POST' al puerto donde el modelo está siendo expuesto con el 'header' de contenido JSON
resp = requests.post(url, headers=headers, data = X_test)

# Imprime el contenido de la respuesta
print(resp.content)

Una alternativa a ejecutar el código de `Python` anterior es realizar la inferencia con un comando `curl` al `localhost`:

Como se puede apreciar, no se trata de un método amigable para realizar las predicciones, y por tanto, crear una imagen de `Docker` y exportar a producción. Por lo tanto, se va a usar otra alternativa.

# Despliegue (producción)

## Docker

De esta forma, se ha creado una aplicación web con `Streamlit` dentro de la carpeta `app` que va a permitir desplegar el modelo de forma más amigable mediante una interfaz gráfica. Una vez se ha creado el código de la aplicación y testeada la misma en local, se crea una imagen de `Docker` con la interfaz mediante un `Dockerfile`:
- En este archivo se copia el código de la aplicación web de `Streamlit`, el archivo `requirements.txt` con las librerías requeridas y el modelo guardado (en un archivo `.pkl` gracias a `MLflow`) al directorio de trabajo de la imagen de `Docker`.
- Se actualizan los paquetes e instalan las librerías requeridas del `requirements.txt`.
- Se expone el puerto `8501`, ya que posteriormente se va a usar para exponer la aplicación web.
- Se ejecuta la aplicación web de `Streamlit` (`app.py`) especificando la `IP` y el `puerto` donde se servirá en el `ENTRYPOINT`.

Tras crear este archivo y configurarlo correctamente, se crea la imagen con el siguiente comando:

<p style="text-align: center;"> docker build -t app-embedded-airbnb:v1.1 . </p>

Mediante el argumento `-t` se especifica el nombre y el `tag` de la imagen en formato `nombre:tag`. Por lo que la imagen creada se llama `app-embedded-airbnb` y el `tag` es `v1.1`. El `.` del final indica que el archivo `Dockerfile` está en el directorio actual (`Airbnb_Madrid`).

Una vez se crea la imagen de `Docker`, ya se puede consultar la aplicación web donde se ha servido el modelo dentro de un contenedor. Para ello se inicia un contenedor, como se ha comentado, a partir de la imagen creada con un comando como el siguiente:

<p style="text-align: center;"> docker container run -itd -p 8501:8501 --name app-streamlit 3ed </p>

Mediante este comando se inicia un contenedor a partir de la imagen creada, enlazando el puerto `8501` de nuestro `localhost` con el puerto `8501` del contenedor (donde se encuentra la aplicación web dentro del contenedor). De esta forma, si se abre en un navegador la dirección `127.0.0.1:8501` (el `localhost`) se accederá a la aplicación web de `Streamlit`, ya que el puerto `8501` de nuestro `localhost` está conectado con el puerto `8501` del contenedor, que es donde se aloja la aplicación web del modelo. Por tanto, para obtener predicciones basta con seleccionar las distintas variables de entrada de la aplicación web.

Después de construir la imagen de Docker, ésta se sube a `Dockerhub` para tenerlo almacenado en un repositorio y poder descargar la imagen posteriormente en cualquier ordenador. Para ello, primero se crea un `tag` a la imagen de `Docker` creada para especificar cual es nuestro usuario `Dockerhub` y cual sería el nombre de la imagen y el `tag` en el repositorio. Por tanto, se ejecuta el siguiente comando:

<p style="text-align: center;"> docker image tag app-embedded-airbnb:v1.1 xxxx/app-embedded-airbnb:v1.1 </p>

Donde `xxxx` es el nombre de la cuenta de `Dockerhub`.

Una vez la imagen ya ha sido `taggeada`, se sube al repositorio:

<p style="text-align: center;"> docker image push xxx/app-embedded-airbnb:v1.1 </p>

## Kubernetes

`Kubernetes` es un sistema de orquestación de contenedores usado para el despliegue de contenedores de `Docker`. Está diseñado para administrar eficientemente y coordinar `clusters` y cargas de trabajo a gran escala en un entorno de producción. Además, ayuda a gestionar servicios contenerizados mediante la automatización en el despliegue.